In [1]:
pip install langchain_community langchain langchain_core langchain_ollama

Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env", override=True)

# Verify
for var in ["OPENAI_API_KEY", "LANGSMITH_API_KEY"]:
    print(var, "✅" if os.getenv(var) else "❌ missing")

OPENAI_API_KEY ✅
LANGSMITH_API_KEY ✅


In [26]:
from langchain_ollama import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document


In [27]:
speech = """Even though you are only a very small speck of ocean...
(put your full text here)
"""
docs = [Document(page_content=speech)]


In [28]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

In [29]:
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever()


In [30]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer based only on the context."),
    ("user", "Question: {question}\nContext: {context}")
])

model = ChatOllama(model="gpt-oss:120b-cloud")   # local LLM
parser = StrOutputParser()
chain = prompt | model | parser

In [33]:
# Direct LLM call without retrieval
question = "What are the key messages of the speech?"

context = """Even though you are only a very small speck of ocean...
(put full speech here)"""

print(chain.invoke({"question": question, "context": context}))


I’d be happy to pull out the key messages for you, but I’ll need the full text of the speech. Could you please provide the complete speech (or the portion you’d like summarized)? Once I have it, I can identify the main points and themes for you.


In [34]:
query = "What are the key messages of the speech?"
docs_context = retriever.get_relevant_documents(query)

# Concatenate top chunks as context
rag_context = "\n".join([d.page_content for d in docs_context])

print(chain.invoke({"question": query, "context": rag_context}))

I’m happy to help pull out the key messages, but I’ll need the actual text of the speech to do that. Could you please provide the full content (instead of the placeholder “(put your full text here)”) so I can review it and summarize the main points for you?
